# Imports

In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate

from sklearn import preprocessing
from sklearn.cluster import KMeans
from sklearn import metrics

import time
from contextlib import contextmanager
import gc
import warnings
import pprint
from IPython.display import clear_output

# Configuration

In [21]:
RANDOM_STATE= 190276
ROOT_INPUT = './data/work/'

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)
sns.set()

# Fonctions

In [22]:
def scale(df: pd.DataFrame, robust = True) -> pd.DataFrame:
    
    if robust:
        scaler = preprocessing.RobustScaler()
    else:
        scaler = preprocessing.StandardScaler()
    columns = df.columns
    index = df.index
    df_scaled = scaler.fit_transform(df)
    
    df_scaled = pd.DataFrame(df_scaled, columns=columns, index=index)
    
    return df_scaled

# Model

In [23]:
def model_kmean(df, n_clusters, random_state = None):
    
    # Selection des types number uniquement
    df_kmean = df.select_dtypes('number')
    
    # Robust scale
    df_scaled = scale(df_kmean)
    
    # Fit
    kmeans = KMeans(n_clusters=n_clusters, random_state=random_state)
    kmeans.fit(df_scaled)
    
    return kmeans

# Load DataFrame

In [24]:
df = pd.read_pickle(ROOT_INPUT + 'rfm.pickle')
df

,recence,frequence,montant,order_purchase_timestamp_max,order_period_year_max,order_period_quarter_max,order_period_month_max
customer_unique_id,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,1938,0.0,129.90,2018-05-10 10:56:27,2018,2018Q2,2018-05
0000b849f77a49e4a4ce2b2a4ca5be3f,1941,0.0,18.90,2018-05-07 11:11:27,2018,2018Q2,2018-05
0000f46a3911fa3c0805444483337064,2364,0.0,69.00,2017-03-10 21:05:03,2017,2017Q1,2017-03
0000f6ccb0745a6a4b88665a16c9f078,2148,0.0,25.99,2017-10-12 20:29:41,2017,2017Q4,2017-10
0004aac84e0df4da2b147fca70cf8255,2115,0.0,180.00,2017-11-14 19:45:42,2017,2017Q4,2017-11
...,...,...,...,...,...,...,...
fffcf5a5ff07b0908bd4e2dbc735a684,2274,0.0,1570.00,2017-06-08 21:00:36,2017,2017Q2,2017-06
fffea47cd6d3cc0a88bd621562a9d061,2089,0.0,64.89,2017-12-10 20:07:56,2017,2017Q4,2017-12
ffff371b4d645b6ecea244b27531430a,2395,0.0,89.90,2017-02-07 15:49:16,2017,2017Q1,2017-02


In [25]:
df_2017 = df[df['order_period_year_max']=='2017']

In [19]:
model_2017 = model_kmean(df_2017, 4, RANDOM_STATE)

In [ ]:
model_2017.